In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from IPython.display import display

In [3]:
e2020 = pd.read_csv("data/2020.csv")
e2020.head()

,candidate,party,region,sex,votes
0,Nana Addo Dankwa Akuffo-Addo,npp,ahafo,M,145584
1,John Dramani Mahama,ndc,ahafo,M,116485
2,Christian Kwabena Andrews,gum,ahafo,M,1493
3,Ivor Kobina Greenstreet,cpp,ahafo,M,194
4,Akua Donkor,gfp,ahafo,F,49


In [4]:
e2024 = pd.read_csv("data/2024.csv")
e2024.head()

,candidate,party,region,sex,votes
0,Mahamudu Bawumia,npp,ahafo,M,113851
1,Daniel Augustus Lartey Jnr.,gcpp,ahafo,M,357
2,Akua Donkor,gfp,ahafo,F,0
3,Christian Kwabena Andrews,gum,ahafo,M,221
4,Kofi Akpaloo,lpg,ahafo,M,70


In [5]:
def get_votes_by_region(df):
    return df[["region", "votes"]].groupby("region").sum()

def get_votes_by_party(df):
    return df[["party", "votes"]].groupby("party").sum()

def get_party_region_votes(df):
    return df[["party", "region", "votes"]]

# 2020 Votes by Region

In [6]:
votes_by_region_20 = get_votes_by_region(e2020)
display(votes_by_region_20)
print(f"Total votes: {votes_by_region_20.sum()['votes']}")

,votes
region,
ahafo,264491
ashanti,2467291
bono,502585
bono east,372408
central,1173439
eastern,1236940
greater accra,2602508
north east,239123
nothern,897731


Total votes: 13119520


# 2024 Votes by Region

In [7]:
votes_by_region_24 = get_votes_by_region(e2024)
display(votes_by_region_24)
print(f"Total votes: {votes_by_region_24.sum()['votes']}")

,votes
region,
ahafo,247615
ashanti,2106108
bono,438486
bono east,347040
central,965551
eastern,961615
greater accra,1977079
north east,249351
nothern,914722


Total votes: 11191152


# 2020 Votes by Party

In [8]:
votes_by_party_20 = get_votes_by_party(e2020)
display(votes_by_party_20)

,votes
party,
apc,7138
cpp,12200
gcpp,3564
gfp,5574
gum,105548
ind,9704
lpg,7743
ndc,6213182
ndp,6549


# 2024 Votes by Party

In [9]:
votes_by_party_24 = get_votes_by_party(e2024)
display(votes_by_party_24)

,votes
party,
apc,17461
cpp,23397
gcpp,16673
gfp,0
gum,16461
ind,121827
lpg,5219
ndc,6328397
ndp,4413


# 2020 Party - Region Votes

In [10]:
party_region_votes_20 = get_party_region_votes(e2020)
display(party_region_votes_20)

,party,region,votes
0,npp,ahafo,145584
1,ndc,ahafo,116485
2,gum,ahafo,1493
3,cpp,ahafo,194
4,gfp,ahafo,49
...,...,...,...
187,lpg,volta,237
188,pnc,volta,220
189,ppp,volta,314
190,ndp,volta,617


# 2024 Party - Region Votes

In [11]:
party_region_votes_24 = get_party_region_votes(e2024)
display(party_region_votes_24)

,party,region,votes
0,npp,ahafo,113851
1,gcpp,ahafo,357
2,gfp,ahafo,0
3,gum,ahafo,221
4,lpg,ahafo,70
...,...,...,...
203,apc,volta,929
204,ind,volta,93
205,ind,volta,98
206,ind,volta,1542


In [12]:
def calculate_percentage_change(change, previous_total):
    percentage = (change / previous_total) * 100

    if percentage >= 0:
        return f"+{percentage:.2f}%"
    else:
        return f"{percentage:.2f}%"

def apply_percentages(datasets, merged_df):
    for i in range(1, len(datasets)):
        year_prev = datasets[i - 1]["year"]
        year_curr = datasets[i]["year"]

        merged_df[year_curr] = merged_df[year_curr].fillna(0).astype(int)
        merged_df[year_prev] = merged_df[year_prev].fillna(0).astype(int)
        
        merged_df["Changes"] = merged_df.apply(
            lambda row: calculate_percentage_change(
                row[year_curr] - row[year_prev],
                row[year_prev]
            ),
            axis=1
        )

        

# Combined Votes

In [13]:
datasets = [
    {
        "year": "2020",
        "regional_votes": votes_by_region_20,
        "party_votes": votes_by_party_20,
    },
    {
        "year": "2024",
        "regional_votes": votes_by_region_24,
        "party_votes": votes_by_party_24,

    }
]

merged_df_regions = None
merged_df_party = None

for dataset in datasets:
    regional_df = dataset["regional_votes"].rename(columns={"votes": dataset["year"]})
    party_df = dataset["party_votes"].rename(columns={"votes": dataset["year"]})

    if merged_df_regions is None:
        merged_df_regions = regional_df
    else:
        merged_df_regions = pd.merge(merged_df_regions, regional_df, on="region", how="outer")

    if merged_df_party is None:
        merged_df_party = party_df
    else:
        merged_df_party = pd.merge(merged_df_party, party_df, on="party", how="outer")

apply_percentages(datasets, merged_df_regions)
apply_percentages(datasets, merged_df_party)

# Changes by Region

In [14]:
display(merged_df_regions)

,2020,2024,Changes
region,,,
ahafo,264491,247615,-6.38%
ashanti,2467291,2106108,-14.64%
bono,502585,438486,-12.75%
bono east,372408,347040,-6.81%
central,1173439,965551,-17.72%
eastern,1236940,961615,-22.26%
greater accra,2602508,1977079,-24.03%
north east,239123,249351,+4.28%
nothern,897731,914722,+1.89%


# Changes by Party

In [15]:
display(merged_df_party)

,2020,2024,Changes
party,,,
apc,7138,17461,+144.62%
cpp,12200,23397,+91.78%
gcpp,3564,16673,+367.82%
gfp,5574,0,-100.00%
gum,105548,16461,-84.40%
ind,9704,121827,+1155.43%
lpg,7743,5219,-32.60%
ndc,6213182,6328397,+1.85%
ndp,6549,4413,-32.62%
